<a href="https://colab.research.google.com/github/mateusduarte-max/Projetos_Python/blob/main/07_Onboarding_distribuicao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
from google.colab import files
import datetime as dt
import numpy as np


# Bibliotecas Google Golab
from google.colab import auth
auth.authenticate_user()

# Importar biblioteca biqquery
from google.cloud import bigquery

# Bibliotecas Google Sheet
import gspread
from oauth2client.client import GoogleCredentials
from gspread_dataframe import set_with_dataframe

from google.colab import drive
drive.mount('/content/drive')

# Autenticação Final
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
# projeto
maga_bigdata = 'maga-bigdata'
#autenticação e select
client = bigquery.Client(project=maga_bigdata)
sql = """select
        o.seller_id                                  as Seller_Id
       ,FORMAT_DATETIME('%d/%m/%Y', o.data_cadastro) as Cadastro
       ,COALESCE(o.plataform,'-')                    as Plataform
       ,COALESCE(o.partner_source,'-')               as Canal
       ,o.qtd_servico                                as Qtd_Servicos_MGLU
       ,COALESCE(o.inconsistencias,'-')              as Inconsistencia
       ,FORMAT_DATETIME('%d/%m/%Y', dt_pedido_teste) as Dt_Publicacao
from
     maga-bigdata.indicadores_qa.onboarding as o
      where
        o.data_cadastro >= '2019-01-01'
     order by o.data_cadastro desc"""

# Select salvo na variavel
labs = client.query(sql).to_dataframe().fillna(0)

# Validação canal
def val_canal(labs):
  if ("-" in labs):
    return 'Distribuição'    
  else:
    return 'Hunting CM e lojas físicas'

labs['Val_Canal'] = labs['Canal'].apply(lambda x: val_canal(x))
labs.head()

In [ ]:
# importe base atendimento
base_atendimento = pd.read_csv('/content/drive/MyDrive/dados_onboarding/base_atendimento.csv').fillna('')
base_atendimento.rename(columns= {'Ticket ID': 'ID do ticket','Subject': 'Assunto','Group name': 'Nome do grupo','Created date': 'Data de criação'},inplace=True)

# importe base atendimento
resolvido_projeto_50 = pd.read_csv('/content/drive/MyDrive/dados_onboarding/resolvidos_projeto_50.csv').fillna('')
resolvido_projeto_50.rename(columns= {'Ticket ID': 'ID do ticket','Subject': 'Assunto','Group name': 'Nome do grupo','Created date': 'Data de criação'},inplace=True)



# importe base atendimento
#base_atendimento = pd.read_csv('/content/drive/MyDrive/dados_onboarding/base_atendimento.csv').fillna('')
# importe base atendimento
#resolvido_projeto_50 = pd.read_csv('/content/drive/MyDrive/dados_onboarding/resolvidos_projeto_50.csv').fillna('')

# junção bases
base_tt = pd.concat([base_atendimento, resolvido_projeto_50])

# Expressão regular - pegar seller
def nome(base_tt):
  extracao = re.match(r"[^[]*\[([^]]*)\]", base_tt).groups()[0]
  return extracao
base_tt['Seller_Id'] = base_tt['Assunto'].apply(nome)

base_tt.head(2)


In [ ]:
# Base consolidada
base_consolidada = pd.merge(base_tt, labs, how='left', on='Seller_Id')
base_consolidada.head(2)


In [ ]:
#Separação bases
base_Hunting_CM_lojas_físicas = base_consolidada.query('Val_Canal=="Hunting CM e lojas físicas"')
base_distribuicao = base_consolidada.query('Val_Canal=="Distribuição"')

In [ ]:
# Caixa Hunting CM e lojas fisicas
Hunting_CM = base_Hunting_CM_lojas_físicas[['ID do ticket']]
Hunting_CM.rename(columns={'ID do ticket': 'id'}, inplace=True)
Hunting_CM['group_id'] = '22000165628'
Hunting_CM['cf_especificao_implantao_de_novas_lojas'] = 'Ipdv'
Hunting_CM['cf_market_place'] = 'Processo'
Hunting_CM_export = Hunting_CM[['id', 'group_id', 'cf_especificao_implantao_de_novas_lojas', 'cf_market_place']]

def input_drive():
  #Abrir planilha e primeira aba
  pagina = gc.open('Caixa Hunting CM e lojas fisicas').sheet1
  # Limpar base
  pagina.clear()
  # Importar dataframe na planilha
  return set_with_dataframe(pagina, Hunting_CM_export, include_index=False)
input_drive()

print("Arquivo atualizado com sucesso!")

In [ ]:
# Validação distribuição

# Função validação inconsistências
def val_inconsistencia(base_distribuicao):
  if ("Pedido Teste" in base_distribuicao):
    return 'Catalogação'
  if ("Serviços Desligados" in base_distribuicao):
    return 'Catalogação'
  elif ("Etapa 4" in base_distribuicao):
    return 'Catalogação'
  elif ("Etapa 1" in base_distribuicao):
    return 'Catalogação'
  elif ("Etapa 3" in base_distribuicao):
    return 'Catalogação'
  elif ("Transportadora" in base_distribuicao):
    return 'Hunting-50'
  elif ("Produtos Inativos" in base_distribuicao):
    return 'Hunting-50'
  elif ("Sem estoque" in base_distribuicao):
    return 'Hunting-50'
  elif ("Certificado" in base_distribuicao):
    return 'Hunting-50'
  elif ("Contrato" in base_distribuicao):
    return 'Hunting-50'
  elif ("Sem produto" in base_distribuicao):
    return 'Hunting-50'
  elif ("-" in base_distribuicao):
    return 'Catalogação'
  else:
    return ''
  
base_distribuicao['Caixa'] = base_distribuicao['Inconsistencia'].apply(lambda x: val_inconsistencia(x))
base_distribuicao.head(3)


In [ ]:
# Caixa Onboarding iPDV - Catalogação Problema
Onboarding_catalogacao_p = base_distribuicao.set_index('Caixa').loc[['Catalogação'], ['ID do ticket', 'Seller_Id']]
#Onboarding_catalogacao_p = base_final.query("Id_grupo=='22000165598'", inplace=False)
Onboarding_catalogacao_p.rename(columns={'ID do ticket': 'id'}, inplace=True)
Onboarding_catalogacao_p.rename(columns={'Seller_Id': 'cf_id_helena331775'}, inplace=True)
Onboarding_catalogacao_p['status'] = 2
Onboarding_catalogacao_p['type'] = 'Implantação de novas lojas'
Onboarding_catalogacao_p['group_id'] = '22000165598'

Onboarding_catalogacao_p['responder_id']= ""                                                                      
Onboarding_catalogacao_p['cf_market_place'] = 'Processo'
Onboarding_catalogacao_p_export = Onboarding_catalogacao_p[['id', 'cf_id_helena331775', 'status','type','group_id', 'responder_id', 'cf_market_place']]
Onboarding_catalogacao_p_export.tail(5)

def input_drive():
  #Abrir planilha e primeira aba
  pagina = gc.open('Caixa Onboarding iPDV - Catalogação Problema').sheet1
  # Limpar base
  pagina.clear()
  # Importar dataframe na planilha
  return set_with_dataframe(pagina, Onboarding_catalogacao_p_export, include_index=False)
input_drive()
print('Tabela atualizada!')

In [ ]:
# Busca dados clusterização

# projeto
maga_bigdata = 'maga-bigdata'
#autenticação e select
client = bigquery.Client(project=maga_bigdata)
sql = """SELECT 
       c.HELLENA_SELLER_CODE as cf_id_helena331775,
       c.CNPJ_FORMATADO as cf_cnpj506663,
       c.TELEFONE as cf_telefone_de_contato503518,
       c.EMAIL as cf_email_de_contato372982,
       c.NOME as cf_nome_de_contato734518,
       --c.GRUPO_COMERCIAL 
     FROM `maga-bigdata.indicadores_qa.ipdv_clusterizacao_novo` as c"""

# Select salvo na variavel
clusterizacao = client.query(sql).to_dataframe().fillna(0)

# Caixa Projeto 50
projeto_50 = base_distribuicao.set_index('Caixa').loc[['Hunting-50'], ['ID do ticket', 'Seller_Id', 'Nome do grupo']]
projeto_50.rename(columns={'Nome do grupo': 'Nome_do_grupo'}, inplace=True)
projeto_50.rename(columns={'ID do ticket': 'id'}, inplace=True)
projeto_50.rename(columns={'Seller_Id': 'cf_id_helena331775'}, inplace=True)
projeto_50['type'] = 'Implantação de novas lojas'
projeto_50['cf_especificao_implantao_de_novas_lojas'] = 'Hunting (Projeto 50)'
projeto_50['status'] = 2
projeto_50['priority'] = 2
projeto_50['group_id'] = '22000165265'
projeto_50['cf_tipo'] = 'Hunting (Projeto 50)'

projeto_50_2 = pd.merge(projeto_50, clusterizacao, how='left', on='cf_id_helena331775')
projeto_50_2['cf_nome_de_contato734518'] = projeto_50_2['cf_nome_de_contato734518'].str.lower()
projeto_50_3 = projeto_50_2.query('Nome_do_grupo!="Hunting (Projeto 50)"').fillna('')
projeto_50_3['cf_market_place'] = 'Processo'
projeto_50_4 = projeto_50_3.query('cf_cnpj506663!=""')

# lista de distribuição
lista_agentes = ['22030653763',
                '22030653821',
                '22030653817',
                '22030499222',
                '22030499164',
                '22016805423',
                '22030653773',
                '22030499329',
                '22030499283']

lista_seller = list(projeto_50_4['cf_id_helena331775'])                
n1 = (len(lista_seller)//len(lista_agentes))
lista_original = lista_agentes * n1
n2 = round(len(lista_seller)-len(lista_original))
lista_complementar = (n2 * list(['22030499283']))
projeto_50_4['responder_id'] = (lista_original + lista_complementar)

def tamanho_telefone(projeto_50_4):
    return len(projeto_50_4['cf_telefone_de_contato503518'])
projeto_50_4['tamanho_tel'] =   projeto_50_4.apply(tamanho_telefone, axis=1)

def separador_telefone(projeto_50_4):
    if projeto_50_4['tamanho_tel'] == 10:
        return projeto_50_4['cf_telefone_de_contato503518'][0:7]+'-'+projeto_50_4['cf_telefone_de_contato503518'][6:]
    elif projeto_50_4['tamanho_tel'] == 11:
        return projeto_50_4['cf_telefone_de_contato503518'][0:8]+'-'+projeto_50_4['cf_telefone_de_contato503518'][7:]

projeto_50_4['cf_telefone_de_contato503518'] =   projeto_50_4.apply(separador_telefone, axis=1)

projeto_50_5 = projeto_50_4[['id', 
                             'cf_id_helena331775', 
                             'type', 
                             'cf_especificao_implantao_de_novas_lojas', 
                             'status', 
                             'priority',
                             'group_id',
                             'cf_tipo',
                             'cf_cnpj506663',
                             'responder_id',
                             'cf_telefone_de_contato503518', 
                             'cf_email_de_contato372982', 
                             'cf_nome_de_contato734518', 
                             'cf_market_place']]

def input_drive():
  #Abrir planilha e primeira aba
  pagina = gc.open('Caixa Onboarding - Projeto 50').sheet1
  # Limpar base
  pagina.clear()
  # Importar dataframe na planilha
  return set_with_dataframe(pagina, projeto_50_5, include_index=False)
input_drive()

print("Arquivo atualizado com sucesso!")



In [ ]:
# Análise serviços 
base_distribuicao['Vali_serviço'] = base_distribuicao['Caixa'].str.contains('Catalogação')
servico = base_distribuicao.query('Vali_serviço==True').fillna('')

servico['Qtd_Servicos_MGLU'] = servico['Qtd_Servicos_MGLU'].astype(int)

ligar_serviço = servico.query('Qtd_Servicos_MGLU<12')
servico_labs = ligar_serviço[['Seller_Id', 'Qtd_Servicos_MGLU']]

def input_drive():
  #Abrir planilha e primeira aba
  pagina = gc.open('Serviços_labs').sheet1
  # Limpar base
  pagina.clear()
  # Importar dataframe na planilha
  return set_with_dataframe(pagina, servico_labs, include_index=False)
input_drive()

print("Arquivo atualizado com sucesso!")